In [2]:
def calculateError(real, predicted):
    correct = 0
    for x in range(len(real)):
        if real[x] == predicted[x]:
            correct += 1
    return correct / len(real)

In [4]:
import os
import pandas as pd
#from https://www.kaggle.com/borismarjanovic/price-volume-data-for-all-us-stocks-etfs removing all stocks with less than 1 year of data
filePath = 'price-volume-data-for-all-us-stocks-etfs/Stocks'
fileList = os.listdir(filePath)
masterDF = pd.DataFrame({'Date': [],
                        'Open': [],
                        'High': [],
                        'Low': [],
                        'Close': [],
                        'Volume': [],
                        'OpenInt': [],
                        'ticker': [],
                        'fiveDayAverage': [],
                        'tenDayAverage': [],
                        'fiveDayWeightedAverage': [],
                        'tenDayWeightedAverage': [],
                        'fiveDaySlopeChange': [],
                        'tenDaySlopeChange': [],
                        'tmmrwChngAsPerc': [],
                        'zScoreOfChange': [],
                        'percentChangeInFiveDays': []},
                  index=[])   
for file in fileList:
    try:
        df = pd.read_csv(filePath + '/' + file, parse_dates = True)
        tickerSymbol = file.split('.txt')[0]
        rowCount = df.shape[0]
        df = df.assign(ticker = [tickerSymbol]* df.shape[0], fiveDayAverage = [0.0]* rowCount, tenDayAverage = [0.0]* rowCount,
                       fiveDayWeightedAverage = [0.0]* rowCount, tenDayWeightedAverage = [0.0]* rowCount,fiveDaySlopeChange = [0.0]* rowCount,
                       tenDaySlopeChange = [0.0]* rowCount,tmmrwChngAsPerc = [0.0]* rowCount,zScoreOfChange = [0.0]*rowCount,
                       percentChangeInFiveDays = [0.0]* rowCount)
        #df['ticker'] = [ticker] * df.shape[0]
        df['fiveDayAverage'] = ((df['Close'] + df['Close'].shift(1) + df['Close'].shift(2) + df['Close'].shift(3) + df['Close'].shift(4)).div(5))
        df['tenDayAverage'] = ((df['Close'] + df['Close'].shift(1) + df['Close'].shift(2) + df['Close'].shift(3) + df['Close'].shift(4) + 
                               (df['Close'].shift(5) + df['Close'].shift(6) + df['Close'].shift(7) + df['Close'].shift(8) + df['Close'].shift(9)).div(10)))
        df['fiveDayWeightedAverage'] = (((df['Close']*5) + (df['Close'].shift(1) * 4) + (df['Close'].shift(2) * 3) + 
                                         (df['Close'].shift(3) * 2) + (df['Close'].shift(4))).div(15))
        df['tenDayWeightedAverage'] = (((df['Close']*10) + (df['Close'].shift(1) * 9) + (df['Close'].shift(2) * 8) + 
                                         (df['Close'].shift(3) * 7) + (df['Close'].shift(4) * 6) + (df['Close'].shift(5) * 5)+
                                         (df['Close'].shift(7) * 3)+ (df['Close'].shift(8) * 2) + (df['Close'].shift(9))).div(55))
        df['fiveDaySlopeChange'] = ((df['Close']  - df['Open'].shift(4)).div(5))
        df['tenDaySlopeChange'] = ((df['Close']  - df['Open'].shift(9)).div(10))        
        df['tmmrwChngAsPerc'] = ((df['Close'].shift(-1) - df['Open'].shift(-1)).div(df['Open'].shift(-1)))
        stdDevOfChangePercent = pd.Series.std(df['tmmrwChngAsPerc'])
        meanOfChangePercent = pd.Series.mean(df['tmmrwChngAsPerc'])
        df['zScoreOfChange'] = (df['tmmrwChngAsPerc'] - meanOfChangePercent).div(stdDevOfChangePercent)
        df['percentChangeInFiveDays'] = ((df['Close'].shift(-5) - df['Close']).div(df['Close']))
        df = df.dropna()
        masterDF = pd.concat([masterDF, df])
    except:
        print("error for: ", file)
# for file in fileList:
#     try:
#         df = pd.read_csv(filePath + '/' + file, parse_dates = True)
#         ticker = file.split('.txt')[0]
#         df['ticker'] = [ticker] * df.shape[0]
#         df['fiveDayAverage'] = ((df['Close'] + df['Close'].shift(1) + df['Close'].shift(2) + df['Close'].shift(3) + df['Close'].shift(4)).div(5))
#         df['tenDayAverage'] = ((df['Close'] + df['Close'].shift(1) + df['Close'].shift(2) + df['Close'].shift(3) + df['Close'].shift(4) + 
#                                (df['Close'].shift(5) + df['Close'].shift(6) + df['Close'].shift(7) + df['Close'].shift(8) + df['Close'].shift(9)).div(10)))
#         df['fiveDayWeightedAverage'] = (((df['Close']*5) + (df['Close'].shift(1) * 4) + (df['Close'].shift(2) * 3) + 
#                                          (df['Close'].shift(3) * 2) + (df['Close'].shift(4))).div(15))
#         df['tenDayWeightedAverage'] = (((df['Close']*10) + (df['Close'].shift(1) * 9) + (df['Close'].shift(2) * 8) + 
#                                          (df['Close'].shift(3) * 7) + (df['Close'].shift(4) * 6) + (df['Close'].shift(5) * 5)+
#                                          (df['Close'].shift(7) * 3)+ (df['Close'].shift(8) * 2) + (df['Close'].shift(9))).div(55))
#         df['fiveDaySlopeChange'] = ((df['Close']  - df['Open'].shift(4)).div(5))
#         df['tenDaySlopeChange'] = ((df['Close']  - df['Open'].shift(9)).div(10))        
#         df['tmmrwChngAsPerc'] = ((df['Close'].shift(-1) - df['Open'].shift(-1)).div(df['Open'].shift(-1)))
#         stdDevOfChangePercent = pd.Series.std(df['tmmrwChngAsPerc'])
#         meanOfChangePercent = pd.Series.mean(df['tmmrwChngAsPerc'])
#         df['zScoreOfChange'] = (df['tmmrwChngAsPerc'] - meanOfChangePercent).div(stdDevOfChangePercent)
#         df['percentChangeInFiveDays'] = ((df['Close'].shift(-5) - df['Close']).div(df['Close']))
#         df = df.dropna()
#         masterDF = pd.concat([masterDF, df])
#     except:
#         print("error for: ", file)
print('finished cleaning')
masterDF.to_csv('masterDF.csv', index=False) 

finished cleaning


In [1]:
import numpy as np
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
import os
import pandas as pd
import pickle

masterDF = pd.read_csv('masterDF.csv', parse_dates = True)
zScoreAnswer = masterDF['zScoreOfChange']
zScoreAnswer = ['long' if x > .5 else 'short' if x < -.7 else 'hold' for x in zScoreAnswer ]
fiveDayChangeAnswer = masterDF['percentChangeInFiveDays']
fiveDayChangeAnswer = ['long' if x > .05 else 'short' if x < -.1 else 'hold' for x in zScoreAnswer ]

masterDF = masterDF.drop(columns=['Date','zScoreOfChange','percentChangeInFiveDays'])
masterList = masterDF.values.to_list()
masterTrainList, masterTestList, zScoreTrainList, zScoreTestList, fiveDayChangeTrainList, fiveDayChangeTestList = train_test_split(
    masterList,zScoreAnswer,fiveDayChangeAnswer,test_size = .25)


zScorePredictor = RandomForestClassifier(min_samples_split = 10)
zScorePredictor.fit(masterTrainList, zScoreTrainList)
fiveDayChangePredictor = RandomForestClassifier(min_samples_split = 10)
fiveDayChangePredictor.fit(masterTrainList, fiveDayChangeTrainList)
pickle.dump(zScorePredictor, open('zScorePredictor.pkl', "wb"))
pickle.dump(fiveDayChangePredictor, open('zScorePredictor.pkl', "wb"))
print('dumped')
zScorePredictions = zScorePredictor.predict(masterTestList)
zScorePredictionAccuracy = calculateError(masterTestList,zScorePredictions)
print('zScorePredictionAccuracy is : ', zScorePredictionAccuracy , '%')
fiveDayPrediction = fiveDayChangePredictor.predict(masterTestList)
fiveDayPredictionAccuracy = calculateError(masterTestList,fiveDayPrediction)
print('fiveDayPredictionAccuracy is : ', fiveDayPredictionAccuracy , '%')

c:\users\kazuya\appdata\local\programs\python\python38\lib\site-packages\sklearn\externals\joblib\__init__.py:15: FutureWarning: sklearn.externals.joblib is deprecated in 0.21 and will be removed in 0.23. Please import this functionality directly from joblib, which can be installed with: pip install joblib. If this warning is raised when loading pickled models, you may need to re-serialize those models with scikit-learn 0.21+.
  warnings.warn(msg, category=FutureWarning)


NameError: name 'pd' is not defined